# **Importing libraries**

In [1]:
!pip install tensorflow_io==0.25.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.4/23.4 MB 33.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 47.0 MB/s eta 0:00:00:00:01
  Attempting uninstall: tensorflow-io-gcs-filesystem
    Found existing installation: tensorflow-io-gcs-filesystem 0.31.0
    Uninstalling tensorflow-io-gcs-filesystem-0.31.0:
      Successfully uninstalled tensorflow-io-gcs-filesystem-0.31.0
  Attempting uninstall: tensorflow_io
    Found existing installation: tensorflow-io 0.31.0
    Uninstalling tensorflow-io-0.31.0:
      Successfully uninstalled tensorflow-io-0.31.0


In [2]:
import tensorflow as tf
import tensorflow.keras.layers as L
from tensorflow.keras.models import Model
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
from random import shuffle
import tensorflow.keras.backend as K

In [3]:
!pip freeze > requirements.txt

# **Loading data**

In [4]:
class DataGeneratorSegmentation(tf.keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, names, batch_size=32, dim=(32,32), n_channels=1,
                 shuffle=True):
        'Initialization'
        self.dim = dim
        self.batch_size = batch_size
        self.names = names
        self.n_channels = n_channels
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.names) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        indexes_batch = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        list_names_temp = [self.names[k] for k in indexes_batch]
        
        # Generate data
        X, Y, W = self.__data_generation(list_names_temp)

        return X, Y, W

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.names))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_names_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        X = np.empty((self.batch_size, *self.dim, 3))
        Y = np.empty((self.batch_size, *self.dim, 1))
        W = np.empty((self.batch_size, *self.dim, 1))

        # Generate data
        for i, name in enumerate(list_names_temp):
            img = cv2.imread(f'/kaggle/input/storms-satellite-imagery/quarter_final/img/{name}')
            mask = cv2.imread(f'/kaggle/input/storms-satellite-imagery/quarter_final/mask/{name}', 0)
            img = cv2.resize(img, (self.dim))/255
            mask = np.reshape(cv2.resize(mask, (self.dim))/255, (*self.dim, 1))
            weights = np.full((224, 224, 1), 0.2)
            x_index, y_index, z_index = np.where(mask==1)
            for x, y, z in zip(x_index, y_index, z_index):
                weights[x, y, z] = 0.8
            W[i,] = weights
            X[i,] = img
            Y[i,] = mask
        return X, Y, W

In [5]:
list_names = os.listdir('/kaggle/input/storms-satellite-imagery/quarter_final/img/')
shuffle(list_names)

In [6]:
train_list_names = list_names[:-200]
params = {'dim': (224, 224), 
          'batch_size': 16, 
          'n_channels': 3, 
          'shuffle': True, 
          'names': train_list_names}

training_generator = DataGeneratorSegmentation(**params)

In [7]:
val_list_names = list_names[-200:]
params = {'dim': (224, 224), 
          'batch_size': 16, 
          'n_channels': 3, 
          'shuffle': True, 
          'names': val_list_names}

validation_generator = DataGeneratorSegmentation(**params)

# **Building the model**

In [8]:
def conv_block(x, num_filters):
    x = L.Conv2D(num_filters, 3, padding="same")(x)
    x = L.BatchNormalization()(x)
    x = L.Activation("relu")(x)
 
    x = L.Conv2D(num_filters, 3, padding="same")(x)
    x = L.BatchNormalization()(x)
    x = L.Activation("relu")(x)
 
    return x

In [9]:
def encoder_block(x, num_filters):
    x = conv_block(x, num_filters)
    p = L.MaxPool2D((2, 2))(x)
    return x, p

In [10]:
def attention_gate(g, s, num_filters):
    Wg = L.Conv2D(num_filters, 1, padding="same")(g)
    Wg = L.BatchNormalization()(Wg)
 
    Ws = L.Conv2D(num_filters, 1, padding="same")(s)
    Ws = L.BatchNormalization()(Ws)
 
    out = L.Activation("relu")(Wg + Ws)
    out = L.Conv2D(num_filters, 1, padding="same")(out)
    out = L.Activation("sigmoid")(out)
 
    return out * s

In [11]:
def decoder_block(x, s, num_filters):
    x = L.UpSampling2D(interpolation="bilinear")(x)
    s = attention_gate(x, s, num_filters)
    x = L.Concatenate()([x, s])
    x = conv_block(x, num_filters)
    return x

In [32]:
def attention_unet(input_shape):
    """ Inputs """
    inputs = L.Input(input_shape)
 
    """ Encoder """
    s1, p1 = encoder_block(inputs, 64)
    s2, p2 = encoder_block(p1, 128)
    s3, p3 = encoder_block(p2, 256)
    s4, p4 = encoder_block(p3, 512)
 
    b1 = conv_block(p4, 1024)
 
    """ Decoder """
    d1 = decoder_block(b1, s4, 512)
    d2 = decoder_block(d1, s3, 256)
    d3 = decoder_block(d2, s2, 128)
    d4 = decoder_block(d3, s1, 64)
 
    """ Outputs """
    outputs = L.Conv2D(1, 1, padding="same", activation="sigmoid")(d4)
 
    """ Model """
    model = Model(inputs, outputs, name="Attention-UNET")
    return model

In [33]:
model = attention_unet((256, 256, 3))

# **Training the model**

In [35]:
import tensorflow.keras.backend as K

def dice_coef(y_true, y_pred, smooth=1):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)


def dice_coef_loss(y_true, y_pred):
    return 1 - dice_coef(y_true, y_pred)

def tversky(y_true, y_pred, smooth=1, alpha=0.7):
    y_true_pos = K.flatten(y_true)
    y_pred_pos = K.flatten(y_pred)
    true_pos = K.sum(y_true_pos * y_pred_pos)
    false_neg = K.sum(y_true_pos * (1 - y_pred_pos))
    false_pos = K.sum((1 - y_true_pos) * y_pred_pos)
    return (true_pos + smooth) / (true_pos + alpha * false_neg + (1 - alpha) * false_pos + smooth)


def tversky_loss(y_true, y_pred):
    return 1 - tversky(y_true, y_pred)


def focal_tversky_loss(y_true, y_pred, gamma=0.75):
    tv = tversky(y_true, y_pred)
    return K.pow((1 - tv), gamma) 

In [36]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
              loss=dice_coef_loss, 
              metrics=[dice_coef])

In [ ]:
history = model.fit(training_generator, 
                    epochs=10,
                    validation_data=validation_generator)

Epoch 1/10
285/285 [==============================] - 334s 941ms/step - loss: 0.1356 - dice_coef: 0.4053 - val_loss: 0.1436 - val_dice_coef: 0.3716
Epoch 2/10
285/285 [==============================] - 264s 925ms/step - loss: 0.1300 - dice_coef: 0.4302 - val_loss: 0.1466 - val_dice_coef: 0.3594
Epoch 3/10
285/285 [==============================] - 264s 924ms/step - loss: 0.1289 - dice_coef: 0.4329 - val_loss: 0.1313 - val_dice_coef: 0.4286
Epoch 4/10
285/285 [==============================] - 264s 924ms/step - loss: 0.1276 - dice_coef: 0.4398 - val_loss: 0.1620 - val_dice_coef: 0.2968
Epoch 5/10
285/285 [==============================] - 263s 923ms/step - loss: 0.1295 - dice_coef: 0.4306 - val_loss: 0.1460 - val_dice_coef: 0.3561
Epoch 6/10
285/285 [==============================] - 263s 921ms/step - loss: 0.1252 - dice_coef: 0.4508 - val_loss: 0.2341 - val_dice_coef: 6.3008e-05
Epoch 7/10
285/285 [==============================] - ETA: 0s - loss: 0.1271 - dice_coef: 0.4425

In [ ]:
plt.plot(history.history["loss"], 'b')
plt.plot(history.history["dice_coef"], 'r')
plt.plot(history.history["val_loss"], 'b', linestyle='dashed')
plt.plot(history.history["val_dice_coef"], 'r', linestyle='dashed')
plt.show()

# **Predict**

In [ ]:
predict = []
for name in os.listdir('/kaggle/input/exam-images/exam_images/'):
    img = cv2.imread(f'/kaggle/input/exam-images/exam_images/{name}')
    img = np.reshape(cv2.resize(img, (256, 256))/255, (1, 256, 256, 3))
    predict.append(model.predict(img, verbose=0))

In [ ]:
#os.makedirs('/kaggle/working/predictions/')
for i, prediction in enumerate(predict):
    ret, thresh = cv2.threshold(prediction[0]*255, 3, 255, 0)
    cv2.imwrite(f"/kaggle/working/predictions/img{i:03d}.jpg", prediction[0]*255)

In [ ]:
import os
import subprocess
from IPython.display import FileLink, display

def download_file(path, download_file_name):
    os.chdir('/kaggle/working/')
    zip_name = f"/kaggle/working/{download_file_name}.zip"
    command = f"zip {zip_name} {path} -r"
    result = subprocess.run(command, shell=True, capture_output=True, text=True)
    if result.returncode != 0:
        print("Unable to run zip command!")
        print(result.stderr)
        return
    display(FileLink(f'{download_file_name}.zip'))

In [ ]:
download_file('/kaggle/working/predictions/', 'out')